In [1]:
import pandas as pd
import numpy as np
import os
import re
from matplotlib import pyplot as plt
import xml
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import requests

In [11]:
os.listdir("./data")

['.ipynb_checkpoints',
 'BOE-A-1996-8930-consolidado_ley propiedad intelectual.pdf',
 'BOE-A-1996-8930-consolidado_ley propiedad intelectual_plain text.txt',
 'db.sqlite3',
 'extraction checkpoint.json',
 'OJDinteractiva-Ultimos-Medios-Auditados_diarios.xls',
 'OJDinteractiva-Ultimos-Medios-Auditados_totales.xls',
 'region_to_media_urls.json',
 'role prompt_keys extraction.txt',
 'spain_audience_total_traffic_2023-07-09 11_08_51.csv',
 'sqldiff.exe',
 'sqlite3.exe',
 'sqlite3_analyzer.exe',
 'test full news_okdiario.txt',
 'test full news_xataka.txt',
 'versiones de prompts para extraer keys de noticias.txt']

In [12]:
traffic = pd.read_csv("./data/spain_audience_total_traffic_2023-07-09 11_08_51.csv")
print(traffic.shape)
traffic.head()

(563, 10)


,Unnamed: 0,name,category,sub_category,daily_unique_ips_avg,total_unique_ips,sequences_of_visits,pages,minutes,seconds
0,0,7DIESACTUALITAT.COM,Noticias e Información,Noticias globales y actualidad,434.0,9303.0,17193.0,28929.0,1.0,39.0
1,1,7TELEVALENCIA.COM,Noticias e Información,Noticias globales y actualidad,7682.0,182966.0,251828.0,312441.0,1.0,4.0
2,2,7TVANDALUCIA.ES,Entretenimiento,Broadcast,1643.0,37572.0,63859.0,180869.0,1.0,25.0
3,3,ACTUALIDADRIOJABAJA.COM,Noticias e Información,Noticias globales y actualidad,2510.0,59854.0,84214.0,106288.0,0.0,38.0
4,4,ACTUALITATVALENCIANA.COM,Noticias e Información,Noticias globales y actualidad,3854.0,98610.0,127268.0,293923.0,1.0,8.0


In [13]:
traffic.category.unique()

array(['Noticias e Información', 'Entretenimiento', 'Hogar y Moda',
       'Automoción',
       'Administraciones Públicas y Organizaciones sin ánimo de lucro',
       'Finanzas, seguros e inversión', 'Familia y estilo de vida',
       'Clasificados, subastas', 'Turismo', 'Medio Ambiente',
       'Educación y empleo', 'eCommerce', 'Información Corporativa',
       'Industria', 'Viajes'], dtype=object)

In [14]:
traffic.sub_category.unique()

array(['Noticias globales y actualidad', 'Broadcast',
       'Noticias de interés especial', 'Hogar y Jardín',
       'Información de automóviles', 'Organizaciones sin ánimo de lucro',
       'Noticias e información financiera',
       'Noticias deportivas y actualidad', 'Gastronomía',
       'Recursos familiares', 'Directorios/Guías locales', nan,
       'Turismo y Hostelería', 'Medio Ambiente',
       'Salud, Fitness y Nutrición', 'Multicategoría',
       'Recursos educativos', 'Música', 'Deportes',
       'Clasificados/Subastas', 'Administraciones Públicas', 'Eventos',
       'Libros', 'Búsqueda de empleo', 'Información Corporativa',
       'Información de viajes y mapas'], dtype=object)

In [50]:
traffic_news_and_info = traffic[(traffic["category"] == "Noticias e Información") & \
                                (traffic["sub_category"] == "Noticias globales y actualidad")].drop(["category", "sub_category"], 
                                                                                                    axis=1)
print(traffic_news_and_info.shape)
traffic_news_and_info.head()

(434, 8)


,Unnamed: 0,name,daily_unique_ips_avg,total_unique_ips,sequences_of_visits,pages,minutes,seconds
0,0,7DIESACTUALITAT.COM,434.0,9303.0,17193.0,28929.0,1.0,39.0
1,1,7TELEVALENCIA.COM,7682.0,182966.0,251828.0,312441.0,1.0,4.0
3,3,ACTUALIDADRIOJABAJA.COM,2510.0,59854.0,84214.0,106288.0,0.0,38.0
4,4,ACTUALITATVALENCIANA.COM,3854.0,98610.0,127268.0,293923.0,1.0,8.0
6,6,AGORAHABLA.COM,320.0,6898.0,10763.0,18814.0,0.0,34.0


In [52]:
medias_1 = traffic_news_and_info.sort_values("daily_unique_ips_avg", ascending=False).head(50).name.reset_index(drop=True)
medias_2 = traffic_news_and_info.sort_values("total_unique_ips", ascending=False).head(50).name.reset_index(drop=True)
medias_3 = traffic_news_and_info.sort_values("sequences_of_visits", ascending=False).head(50).name.reset_index(drop=True)
medias_4 = traffic_news_and_info.sort_values("pages", ascending=False).head(50).name.reset_index(drop=True)
pd.concat([medias_1, 
           medias_2, 
           medias_3, 
           medias_4], axis=1, ignore_index=True)

,0,1,2,3
0,ELESPANOL.COM,ELESPANOL.COM,ELESPANOL.COM,ELESPANOL.COM
1,OKDIARIO.COM,OKDIARIO.COM,OKDIARIO.COM,OKDIARIO.COM
2,ELNACIONAL.CAT,LA RAZON.ES,ELNACIONAL.CAT,ELNACIONAL.CAT
3,LA RAZON.ES,ELDIARIO.ES,LA RAZON.ES,LA RAZON.ES
4,ELDIARIO.ES,ELNACIONAL.CAT,ELDIARIO.ES,ELDIARIO.ES
5,PUBLICO.ES,PUBLICO.ES,PUBLICO.ES,ELDEBATE.COM
6,ELDEBATE.COM,ELDEBATE.COM,ELDEBATE.COM,PUBLICO.ES
7,LEVANTE-EMV.COM,LEVANTE-EMV.COM,LEVANTE-EMV.COM,LEVANTE-EMV.COM
8,DIARIO DE SEVILLA,DIARIO DE SEVILLA,DIARIO DE SEVILLA,DIARIO DE SEVILLA
9,THEOBJECTIVE.COM,THEOBJECTIVE.COM,THEOBJECTIVE.COM,NOTICIAS DE NAVARRA
